In [1]:
import pygmo as pg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# ф-я Розенброка 
class rosenbrock_problem:
    def __init__(self, dim=10): # dim - размерность задачи
        self.dim = dim
    
    def fitness(self, x):
        # ф-я Розенброка: f(x) = sum(100*(x[i+1] - x[i]^2)^2 + (1 - x[i])^2) гдне i = 1, ..., n-1
        return [sum(100 * (x[i+1] - x[i]**2)**2 + (1 - x[i])**2 for i in range(self.dim - 1))]

    def get_bounds(self): # ограничения
        return ([-5] * self.dim, [5] * self.dim)

# ф-я Сферы
class sphere_problem:
    def __init__(self, dim=10):
        self.dim = dim 
    
    def fitness(self, x):
        # Ф-я сферы: f(x) = sum(x[i]^2) где i = 1, ..., n
        return [sum(xi**2 for xi in x)]

    def get_bounds(self):
        return ([-5] * self.dim, [5] * self.dim)

In [3]:
# п-рыы для моделирования отжига (Simulated Annealing)
Ts = 10.0  # начадбная температура
Tf = 0.1   # конечная температура
n_T_adj = 10  # к-во шагов для корректировки температуры
n_range_adj = 1  # к-во шагов для корректировки диапазона
bin_size = 20  # размер бина
start_range = 1.0  # начальный диапазон
seed = 42  # сид для генератора случайных чисел

# Алгоритмы
algo_de = pg.algorithm(pg.de(gen=100))  # диф эволюция
algo_pso = pg.algorithm(pg.pso(gen=100))  # оптимизация роя частиц
algo_sa = pg.algorithm(pg.simulated_annealing(Ts=Ts, Tf=Tf, n_T_adj=n_T_adj, 
                                              n_range_adj=n_range_adj, bin_size=bin_size, 
                                              start_range=start_range, seed=seed))  # моделирование отжига


In [4]:
# ф-я для запуска алгоритма и сбора результатов
def run_algorithm(algorithm, problem, iterations=3):
    results = []
    for _ in range(iterations):
        pop = pg.population(problem, 50)  # размер популяции 50
        result = algorithm.evolve(pop)
        results.append(result.get_f()[0])  # сохраняем мин значение ф-и
    return results


In [5]:
def optimize_and_compare():
    # сравнение
    functions = [rosenbrock_problem(dim=10), sphere_problem(dim=10)]
    function_names = ["Rosenbrock", "Sphere"]
    algorithms = [algo_de, algo_pso, algo_sa]
    algorithm_names = ["DE", "PSO", "SA"]

    # таблица сравнений
    results_df = pd.DataFrame(columns=["Function", "Algorithm", "Run 1", "Run 2", "Run 3", "Average"])

    # прохождение по функциям и алгоритмам
    for fn, fn_name in zip(functions, function_names):
        for algo, algo_name in zip(algorithms, algorithm_names):
            results = run_algorithm(algo, pg.problem(fn))
            avg_result = np.mean(results)
            result_dict = {
                "Function": fn_name,
                "Algorithm": algo_name,
                "Run 1": results[0],
                "Run 2": results[1],
                "Run 3": results[2],
                "Average": avg_result
            }
            results_df = pd.concat([results_df, pd.DataFrame([result_dict])], ignore_index=True)
    
    return results_df


In [6]:
results_table = optimize_and_compare()
print(results_table)

C:\Users\gibutakov\AppData\Local\Temp\ipykernel_7152\4025862461.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([result_dict])], ignore_index=True)


     Function Algorithm                  Run 1                   Run 2  \
0  Rosenbrock        DE   [217.60274502935997]    [165.48207443944304]   
1  Rosenbrock       PSO    [8.941981287118564]    [14.323720318446938]   
2  Rosenbrock        SA    [107693.0257744942]     [84963.13822794202]   
3      Sphere        DE   [0.7925768868882535]    [0.7202601754880197]   
4      Sphere       PSO  [0.07171388925406148]  [0.015467191044267314]   
5      Sphere        SA    [91.89934774737178]     [76.06496335099548]   

                    Run 3        Average  
0    [108.34064387908377]     163.808488  
1    [11.972639572208992]      11.746114  
2     [121171.1506526615]  104609.104885  
3    [1.0968608314270119]       0.869899  
4  [0.060445009638395004]       0.049209  
5     [83.84994002430757]      83.938084  


В этой лабораторной работе реализовали алгоритмы оптимизации для двух разных тестовых функций, сравнили их производительность.